In [2]:
from functools import total_ordering
import streamlit as st
import pandas as pd
import numpy as np

In [3]:
state_name = pd.read_csv("states.csv")

CSS to inject contained in a string

hide_dataframe_row_index = """
            <style>
            .row_heading.level0 {display:none}
            .blank {display:none}
            </style>
            """


Inject CSS with Markdown

In [ ]:
st.markdown(hide_dataframe_row_index, unsafe_allow_html=True)

Household Profile

In [ ]:
with st.sidebar:
	st.header("Household Profile")
	adults = st.slider('Adults', min_value=1, max_value=5, value=None, step=1)  # slider widget to select family members
	st.write('Number of Adults in the Household -', adults)
	children = st.slider('Children', min_value=0, max_value=5, value=None, step=1)  # slider widget to select family members
	st.write('Number of Children in the Household -', children)
	st.write('*Total number of persons in the Household* -', adults+children)
	area_select =  st.selectbox('Area Type',('Urban','Rural'))
	econ_status_select= st.selectbox('Socio-Economic Status',('Lower','Middle','Upper'))
	state_select = st.selectbox('State',state_name)

Primary Cooking Fuel Selection

 edits required - urban selection - fuels available - LPG and PNG<br>
 rural selection - fuels available - Firewood, Biogas and LPG

In [ ]:
with st.expander('1\) Select Baseline Cooking Scenario'):
	#st.header('Baseline Scenario')
	df_base_cf = pd.read_excel("cook-fuel-stove.xlsx",sheet_name="baseline")
	df_base_cf1 = df_base_cf["Fuel"].drop_duplicates()
	bpf_select = st.selectbox('Primary Cooking Fuel',(df_base_cf1))
	mask_bf_stv = df_base_cf['Fuel'].isin([bpf_select])
	df_base_cf_st = df_base_cf[mask_bf_stv].iloc[:,1]
	prst_select = st.selectbox('Primary Cookstove', (df_base_cf_st))
	prst_stack_percent = st.slider('% of cooking done with primary cookstove', min_value = 50, max_value = 100, step = 10)

Electric Cooking Selection

In [ ]:
with st.expander('2\) Select Electric Cooking Scenario'):
	#st.header('e-Cooking Scenario')
	df_elec_cf = pd.read_excel("cook-fuel-stove.xlsx",sheet_name="e-cooking")
	df_elec_cf1 = df_elec_cf["Fuel"].drop_duplicates()
	epf_select = st.selectbox('Primary Electricity Source', (df_elec_cf1))
	mask_ef_stv = df_elec_cf['Fuel'].isin([epf_select])
	df_elec_cf_st = df_elec_cf[mask_ef_stv].iloc[:,1]
	est_sel = st.selectbox('Primary e-cooking equipment',(df_elec_cf_st))
	est_stack_percent = st.slider('% of total household cooking to be done with the e-cooking appliance', min_value = 50, max_value = 100, step = 10)
	

Cooking Pattern

In [ ]:
df_region = pd.read_excel("region-cooking.xlsx")

In [ ]:
with st.expander('3\) Cooking Pattern'):
	#st.markdown('Region & Cuisine')
	mask = df_region['States'].str.contains(state_select, case=False, na=False)
	st.dataframe(df_region[mask].iloc[:,0:1],width=120)
	st.table(df_region[mask].iloc[:,2:3])

In [ ]:
	st.markdown('Daily Cooking Pattern')
	df_dail_cook = df_region.iloc[:3,4:]
	st.dataframe(df_dail_cook)

Results - Cooking Techno-economic analysis

In [ ]:
with st.expander('4\) Results  \- Cookstove Characteristics'):
	cst_var  = {'Variable':['Type','Life','Thermal Efficiency', 'Capex', 'Overheads', 'Total Cost'],
	'Units':['-','Years','%','INR','INR/year','INR'],
	'Baseline':[prst_select,'-','-','-','-','-'],
	'e-Cooking':[est_sel,'-','-','-','-','-'],
	'Delta':['-','-','-','-','-','-']
	}
	df_cst_var = pd.DataFrame(cst_var)
	st.dataframe(df_cst_var)

In [ ]:
with st.expander('5\) Results  \- Energy Consumption'):
	encons_var  = {'Variable':['Daily cooking duration', 'Hourly consumption', 'Daily consumption', 'Annual consumption'],
	'Units':['hours/day','kWh/hour','kWh/day','kWh/year'],
	'Baseline':['-','-','-','-'],
	'e-Cooking':['-','-','-','-'],
	'Delta':['-','-','-','-']
	}
	df_encons_var = pd.DataFrame(encons_var)
	st.dataframe(df_encons_var)

In [ ]:
with st.expander('6\) Results  \- Energy Cost'):
	encost_var  = {'Variable':['Unit cost', 'Opex', 'Overheads', 'Total cost'],
	'Units':['INR/kWh','INR/year','INR/year','INR/year'],
	'Baseline':['-','-','-','-'],
	'e-Cooking':['-','-','-','-'],
	'Delta':['-','-','-','-']
	}
	df_encost_var = pd.DataFrame(encost_var)
	st.dataframe(df_encost_var)

In [ ]:
with st.expander('7\) Results  \- Financing'):
	fin_var  = {'Variable':['Annual Income of HH','Payback period','Unit carbon emission','Total carbon emissions (FS+EI*)','Unit cost of carbon credit (CC)','Total CC revenue possible'],
	'Units':['INR','years','MtCO2eq./kWh','MtCO2eq./year','INR/MtCO2eq.','INR/year'],
	'Baseline':['-','-','-','-','-','-'],
	'e-Cooking':['-','-','-','-','-','-'],
	'Delta':['-','-','-','-','-','-']
	}
	df_fin_var = pd.DataFrame(fin_var)
	st.dataframe(df_fin_var)
	st.markdown('FS - Fuel Switch, EI - Efficiency Improvement')

In [ ]:
with st.expander('8\) Results  \- Health Impacts'):
	hlt_var  = {'Variable':['Daily IHAP (PM2.5)','Annual IHAP (PM2.5)','Health Hazard','mortality reduction'],
	'Units':['μg/m3','μg/m3','-','% or no.'],
	'Baseline':['-','-','-','-'],
	'e-Cooking':['-','-','-','-'],
	'Delta':['-','-','-','-']
	}
	df_hlt_var = pd.DataFrame(hlt_var)
	st.dataframe(df_hlt_var)

1 - create a data frame with the variables<br>
2 - append the units to the data frame<br>
3 - create baseline dataframe - with default values + based on user input values<br>
4 - append to the previous table created<br>
5 - create electric cooking dataframe<br>
6 - append to the previous table<br>
7 - create difference dataframe b/w baseline and electric cooking<br>
8 - append difference df to the main df